In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `~/PhD/CMPBP`


In [2]:
using LinearAlgebra, Statistics, Revise
includet("../src/observables.jl")
includet("../src/truncation.jl")
includet("../src/glauber.jl")
includet("../src/cmpbp.jl")

## Lagrange ascent

In [3]:
function build_S(A,B; q=size(A, 1), da=size(A,4), db=size(B,4))
    sum(kron(E(z,zz;q), I(q), I(da), I(db)) for z in 1:q, zz in 1:q if z≠zz) + 
        sum(kron(E(z,z;q), E(x,x;q), kron(A[x,x,z,:,:],I(db))+kron(I(da),B[x,x,z,:,:])) for z in 1:q, x in 1:q) +
        sum(kron(E(z,z;q), E(x,y;q), A[x,y,z,:,:], B[x,y,z,:,:]) for z in 1:q, x in 1:q, y in 1:q if y≠x)
end

function eigmax_S(A,B)
    q = size(A, 1)
    da, db = size(A,4), size(B,4)
    size(A,2)==size(A,3)==size(B,1)==size(B,2)==size(B,3)==q || error("Incompatible dimensions")
    (size(A,5)==da && size(B,5)==db) || error("Incompatible dimensions")

    S = build_S(A,B; q,da,db)
    maximum(real, eigen(S).values)
    # eigen(S)
end

fidelity_eig(A,B) = eigmax_S(A,B) - 0.5 * (eigmax_S(A,A) + eigmax_S(B,B))

fidelity_eig (generic function with 1 method)

In [4]:
q = 2
da, db = 5, 5

B = rand(q,q,q,da,da)
A = rand(q,q,q,db,db)
# A = copy(B) .+ 0.1.*rand.()
t = Truncator(A,B)

@show marginals(B) marginals(A) fidelity(A,B; maxiter=10^6) fidelity_eig(A,B);

marginals(B) = ComplexF64[0.49637322170347814 + 3.161629217700206e-17im, 0.5036267782965218 - 3.161629217700206e-17im]
marginals(A) = ComplexF64[0.5135004237456954 + 5.3287743622192875e-18im, 0.4864995762543047 - 5.3287743622192906e-18im]
fidelity(A, B; maxiter = 10 ^ 6) = -0.009335742466259234
fidelity_eig(A, B) = -0.009335742466264563


In [5]:
ascent!(A, B, t; maxiters=[1000], ηs=[1e-4])
@show marginals(B) marginals(A) fidelity_eig(A,B);

marginals(B) = ComplexF64[0.49637322170347814 + 3.161629217700206e-17im, 0.5036267782965218 - 3.161629217700206e-17im]
marginals(A) = ComplexF64[0.5058359255505653 + 5.698969122021769e-17im, 0.4941640744494346 - 5.698969122021769e-17im]
fidelity_eig(A, B) = -0.0010293252391786467


## Augmented Lagrangian method

In [6]:
e(x; q=2) = [i==x for i in 1:q]

function aug_lagrangian(A, B, Q, P, Q1, P1, λ, λ1; μ=-1e2, verbose=false)
    q = size(A, 1)
    da, db = size(A,4), size(B,4)

    S = build_S(A,B; q,da,db)
    S1 = build_S(A,A; q,da,db=da)
    vecQ = sum(kron(e(z;q), e(x;q), vec(Q[x,z,:,:])) for z in 1:q, x in 1:q)
    vecQ1 = sum(kron(e(z;q), e(x;q), vec(Q1[x,z,:,:])) for z in 1:q, x in 1:q)
    vecP = sum(kron(e(z;q), e(x;q), vec(P[x,z,:,:])) for z in 1:q, x in 1:q)
    vecP1 = sum(kron(e(z;q), e(x;q), vec(P1[x,z,:,:])) for z in 1:q, x in 1:q)

    Sq = S*vecQ - λ*vecQ
    S1q1 = S1*vecQ1 - λ1*vecQ1

    if verbose
        @show Sq'Sq S1q1'S1q1 vecP'Sq vecP1'S1q1
    end
    return λ - λ1/2 + μ/2 * (Sq'Sq + S1q1'S1q1) + vecP'Sq + vecP1'S1q1
end

aug_lagrangian (generic function with 1 method)

In [13]:
q = 2
da, db = 5, 5

A = rand(q,q,q,da,da)
B = rand(q,q,q,db,db)
# B = copy(A) .+ 1-1 .* rand.()
@show fidelity_eig(A,B)

t = Truncator(A,B);

fidelity_eig(A, B) = -0.7616218521629694


In [14]:
Q, P, Q1, P1 = t.Q, t.P, t.Q1, t.P1
Qold, Pold, Q1old, P1old = t.Qold, t.Pold, t.Q1old, t.P1old
simQ, simQ1, X = t.simQ, t.simQ1, t.X

maxiter_pow = 10^5
tol_pow = 1e-20

findeigen_l!(P, A, B; maxiter_pow, tol_pow, q,da,db, X=simQ, Pold)
findeigen_r!(Q, A, B; maxiter_pow, tol_pow, q,da,db, X=simQ, Qold)
findeigen_l!(P1, A, A; maxiter_pow, tol_pow, q,da,db=da, X=simQ1, Pold=P1old)
findeigen_r!(Q1, A, A; maxiter_pow, tol_pow, q,da,db=da, X=simQ1, Qold=Q1old)
n = dotprod(P, Q)
P ./= n
n1 = dotprod(P1, Q1)
P1 ./= -2n1

Qold .= Q
Pold .= P
Q1old .= Q1
P1old .= P1

apply!(Qold,A,B)
apply_dag!(Pold,A,B)
apply!(Q1old,A,A)
apply_dag!(P1old,A,A)

t.λ = mean(Qold./Q)
t.λ1 = mean(Q1old./Q1)
Qold .= Q
Q1old .= Q1
;

maxiter_pow = 100000
maxiter_pow = 100000
maxiter_pow = 100000


In [21]:
derλ, derλ1, derQ, derQ1, derA = aug_lagrange!(A,B, t; μ=-1e1, ω_stop=1e-3, maxiter=10^2, maxiter_grad=10^2, ηA=1e-3, ηQ=1e-3, ω=1e-3, tol=1e-3, τ=1.2, α=0.5, β=0.8, verbose=true);

ϵC = 0.059491343867419506
Gradient ascent not converged with 0.009635105637996626
ϵC = 0.08251208187633191
Gradient ascent not converged with 0.049820053757181654
ϵC = 0.07019856787584904
Gradient ascent not converged with 0.008807196263624265
ϵC = 0.058205916020392594
Gradient ascent not converged with 0.0027810114560146513
ϵC = 0.048455866600364064
Gradient ascent not converged with 0.002150441419949165
ϵC = 0.04045724421247072
ϵC = 0.03892500948683268
ϵC = 0.034755740776114555
ϵC = 0.02969279091697303
ϵC = 0.02445918248048783
ϵC = 0.01969251896298013
ϵC = 0.015701178794986975
ϵC = 0.014271551317587186
ϵC = 0.010282468845990798
ϵC = 0.007928013493702526
ϵC = 0.007409239346329605
ϵC = 0.005198695720913815
ϵC = 0.005788449288703128
ϵC = 0.003875599681906877
ϵC = 0.003402771095273558
ϵC = 0.0027914072101086524
ϵC = 0.0022389817124466387
ϵC = 0.0023651677862657023
ϵC = 0.0021349279664509783
ϵC = 0.0018604251280315489
ϵC = 0.002253054565305287
ϵC = 0.00551661487712343
ϵC = 0.0022968213111

In [22]:
fidelity_eig(A,B)

-0.003909848520857295

In [23]:
@show marginals(B) marginals(A);

marginals(B) = ComplexF64[0.518482509090113 - 1.004670283586453e-16im, 0.4815174909098871 + 1.004670283586453e-16im]
marginals(A) = ComplexF64[0.5157589284791987 - 1.4880314406636382e-17im, 0.4842410715208013 + 1.4880314406636382e-17im]


In [9]:
using ForwardDiff

autoderA = ForwardDiff.gradient(A->aug_lagrangian(A,B,t.Q,t.P,t.Q1,t.P1,t.λ,t.λ1), A)
autoderQ = ForwardDiff.gradient(Q->aug_lagrangian(A,B,Q,t.P,t.Q1,t.P1,t.λ,t.λ1), t.Q)
autoderP = ForwardDiff.gradient(P->aug_lagrangian(A,B,t.Q,P,t.Q1,t.P1,t.λ,t.λ1), t.P)
autoderQ1 = ForwardDiff.gradient(Q1->aug_lagrangian(A,B,t.Q,t.P,Q1,t.P1,t.λ,t.λ1), t.Q1)
autoderP1 = ForwardDiff.gradient(P1->aug_lagrangian(A,B,t.Q,t.P,t.Q1,P1,t.λ,t.λ1), t.P1)
autoderλ = ForwardDiff.derivative(λ->aug_lagrangian(A,B,t.Q,t.P,t.Q1,t.P1,λ,t.λ1), t.λ)
autoderλ1 = ForwardDiff.derivative(λ1->aug_lagrangian(A,B,t.Q,t.P,t.Q1,t.P1,t.λ,λ1), t.λ1)
;